In [4]:
pip install pennylane

In [5]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from google.colab import files
import io
import pandas as pd

In [6]:
uploaded1 = files.upload()

Saving mock_train_set.csv to mock_train_set.csv


In [22]:
#Reading the given training csv file into a data frame
data = pd.read_csv(io.BytesIO(uploaded1['mock_train_set.csv']))
#Now converting data frame into a numpy array using df.values
train=data.values
#print (train)

X = np.array(train[:, :-1], requires_grad=False)
print ((X))
Y = np.array(train[:, -1], requires_grad=False)
Y = 2*Y - np.ones(len(Y))
#print (X)

[[2.78926e+03 1.00000e+03 1.00000e+01 2.00000e+01]
 [4.04001e+03 1.00000e+06 1.00000e+00 1.00000e+00]
 [2.93120e+03 1.00000e+04 1.00000e+04 4.00000e+01]
 ...
 [4.18281e+03 1.00000e+00 1.00000e+00 6.50000e+01]
 [3.11375e+03 1.00000e+04 1.00000e+02 1.00000e+00]
 [4.56757e+03 1.00000e+04 1.00000e+05 9.00000e+01]]


In [23]:
dev = qml.device("default.qubit", wires=4)

In [24]:

def layer(W):
    qml.RX(W[0, 0], wires=0)
    qml.RX(W[0, 1], wires=0)
    qml.RX(W[0, 2], wires=0)
    qml.RX(W[1, 0], wires=1)
    qml.RX(W[1, 1], wires=1)
    qml.RX(W[1, 2], wires=1)
    qml.RX(W[2, 0], wires=2)
    qml.RX(W[2, 1], wires=2)
    qml.RX(W[2, 2], wires=2)
    qml.RX(W[3, 0], wires=3)
    qml.RX(W[3, 1], wires=3)
    qml.RX(W[3, 2], wires=3)
    qml.RZ(W[0, 0], wires=0)
    qml.RZ(W[0, 1], wires=0)
    qml.RZ(W[0, 2], wires=0)
    qml.RZ(W[1, 0], wires=1)
    qml.RZ(W[1, 1], wires=1)
    qml.RZ(W[1, 2], wires=1)
    qml.RZ(W[2, 0], wires=2)
    qml.RZ(W[2, 1], wires=2)
    qml.RZ(W[2, 2], wires=2)
    qml.RZ(W[3, 0], wires=3)
    qml.RZ(W[3, 1], wires=3)
    qml.RZ(W[3, 2], wires=3)
    qml.RY(W[0, 0], wires=0)
    qml.RY(W[0, 1], wires=0)
    qml.RY(W[0, 2], wires=0)
    qml.RY(W[1, 0], wires=1)
    qml.RY(W[1, 1], wires=1)
    qml.RY(W[1, 2], wires=1)
    qml.RY(W[2, 0], wires=2)
    qml.RY(W[2, 1], wires=2)
    qml.RY(W[2, 2], wires=2)
    qml.RY(W[3, 0], wires=3)
    qml.RY(W[3, 1], wires=3)
    qml.RY(W[3, 2], wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
def statepreparation(x):
    qml.AngleEmbedding(features=x, wires=range(4), rotation='Z')
    qml.Hadamard(0)
    

In [19]:
@qml.qnode(dev)
def circuit(weights, x):

    statepreparation(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

In [11]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [12]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [13]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [14]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights,bias, x) for x in X]
    return square_loss(Y, predictions)

In [39]:
np.random.seed(0)
num_qubits = 4
num_layers = 6
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]

 [[ 0.02269755 -0.01454366  0.00045759]
  [-0.00187184  0.01532779  0.01469359]
  [ 0.00154947  0.00378163 -0.00887786]
  [-0.01980796 -0.00347912  0.00156349]]

 [[ 0.01230291  0.0120238  -0.00387327]
  [-0.00302303 -0.01048553 -0.01420018]
  [-0.0170627   0.01950775 -0.00509652]
  [-0.00438074 -0.01252795  0.0077749 ]]

 [[-0.01613898 -0.0021274  -0.00895467]
  [ 0.00386902 -0.00510805 -0.01180632]
  [-0.00028182  0.00428332  0.00066517]
  [ 0.00302472 -0.00634322 -0.00362741]]

 [[-0.0067246  -0.00359553 -0.00813146]
  [-0.01726283  0.00177426 -0.00401781]
  [-0.01630198  0.00462782 -0.00907298]
  [ 0.00051945  0.00729091  0.00128983]]] 0.0


In [41]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 3

In [42]:
weights = weights_init
bias = bias_init
for it in range(10):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    #print (X_batch)
    Y_batch = Y[batch_index]
    #print (Y_batch)
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

Iter:     1 | Cost: 1.7537068 | Accuracy: 0.4766667 
Iter:     2 | Cost: 1.1955090 | Accuracy: 0.5233333 
Iter:     3 | Cost: 2.3393624 | Accuracy: 0.4766667 
Iter:     4 | Cost: 1.0095663 | Accuracy: 0.4766667 
Iter:     5 | Cost: 1.3053505 | Accuracy: 0.4766667 
Iter:     6 | Cost: 1.1883827 | Accuracy: 0.4766667 
Iter:     7 | Cost: 1.8977823 | Accuracy: 0.5233333 
Iter:     8 | Cost: 1.3684752 | Accuracy: 0.4766667 
Iter:     9 | Cost: 1.6650727 | Accuracy: 0.4766667 
Iter:    10 | Cost: 1.0005680 | Accuracy: 0.5233333 


In [30]:
upload2=files.upload()

Saving mock_test_set.csv to mock_test_set.csv


In [32]:
data_1= pd.read_csv(io.BytesIO(upload2['mock_test_set.csv']))
test=data_1.values
#print (train)

X_test = np.array(test[:, :-1], requires_grad=False)
Y_test = np.array(test[:, -1], requires_grad=False)
Y_test = 2*Y_test - np.ones(len(Y_test))
#print (X_test)

In [43]:
# Compute accuracy
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]
acc = accuracy(Y_test, predictions)

print(
    " Cost: {:0.7f} | Accuracy: {:0.7f} ".format( cost(weights, bias, X, Y), acc)
)

 Cost: 1.0005680 | Accuracy: 0.4833333 
